In [1]:
#required imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from nltk import ne_chunk, pos_tag, word_tokenize
import nltk
import string
import collections
import re
from os import path
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk import bigrams
from nltk import ngrams
from nltk.tree import Tree
from nltk.tag import StanfordNERTagger
import string
import os
os.environ['CLASSPATH'] = "./stanford-ner/"

#local helper file import
import nlp_helper as nlph

In [2]:
raws = pd.read_csv('hillary-clinton-emails/Emails.csv',usecols=['ExtractedSubject','ExtractedBodyText','RawText'])
raws.head()
raws.drop(['RawText'], axis= 1,inplace=True)
body = pd.DataFrame()
body['text'] = raws.apply(nlph.concat_subj_txt, axis=1)
body.head(10)

,text
0,FW: Wow
1,"B6\nThursday, March 3, 2011 9:45 PM\nH: Latest..."
2,Re: Chris StevensThx
3,FVV: Cairo Condemnation - Final
4,"H <hrod17@clintonemail.com>\nFriday, March 11,..."
5,Meet The Right Wing Extremist Behind Anti-Musl...
6,"FW: Anti-Muslim film director in hiding, follo..."
7,"H <hrod17@clintonemail.corn>\nFriday, March 11..."
8,FVV: Secretary's remarksFYI
9,"more on LibyaB6\nWednesday, September 12, 2012..."


In [3]:
if 0:    
    with_loc = pd.DataFrame()
    with_loc['text'] = body['text']
    st = StanfordNERTagger('./stanford-ner/classifiers/english.all.3class.distsim.crf.ser.gz') 
    with_loc['locations'] = with_loc.apply(lambda row: [token for (token,label) in st.tag(word_tokenize(row.text)) if label=='LOCATION'], axis=1)

In [4]:
if 0:
    with_loc.to_csv('text_locations.csv', sep=',', encoding='utf-8')

In [5]:
if 0:
    len(with_loc)

In [6]:
if 0:
    with_loc.head(20)

Using GoogleMaps API (see nlp_helper file), we query the country corresponding to every locations mentionned in each emails. We thus transform our column locations by mapping its locations to their corresponding countries. 

In [7]:
with_loc = pd.read_csv('text_locations.csv', usecols=['text', 'locations'])
with_loc.head()

,text,locations
0,FW: Wow,[]
1,"B6\r\r\nThursday, March 3, 2011 9:45 PM\r\r\nH...","['Syria', 'libya']"
2,Re: Chris StevensThx,[]
3,FVV: Cairo Condemnation - Final,['Cairo']
4,"H <hrod17@clintonemail.com>\r\r\nFriday, March...",['Syria']


We loaded the table containing the detected locations. We now want to map each location to a corresponding country. 

Too much queries needed to use the googlemaps api. 

We decide to use geonames.

In [8]:
#Mapping using googlemaps API

if 0:
    with_loc['locations'] = with_loc.apply(lambda row: list(set(nlph.locations_2_countries(row.locations))), axis=1)
    with_loc.head(20)

In [9]:
#Mapping using geonames API
nlph.geonames('Cairo')

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<geonames>
<totalResultsCount>23</totalResultsCount>
<code>
<postalcode>26337</postalcode>
<name>Cairo</name>
<countryCode>US</countryCode>
<lat>39.23249</lat>
<lng>-81.15543</lng>
<adminCode1>WV</adminCode1>
<adminName1>West Virginia</adminName1>
<adminCode2>085</adminCode2>
<adminName2>Ritchie</adminName2>
<adminCode3/>
<adminName3/>
</code>
<code>
<postalcode>65239</postalcode>
<name>Cairo</name>
<countryCode>US</countryCode>
<lat>39.51141</lat>
<lng>-92.44003</lng>
<adminCode1>MO</adminCode1>
<adminName1>Missouri</adminName1>
<adminCode2>175</adminCode2>
<adminName2>Randolph</adminName2>
<adminCode3/>
<adminName3/>
</code>
<code>
<postalcode>68824</postalcode>
<name>Cairo</name>
<countryCode>US</countryCode>
<lat>41.00018</lat>
<lng>-98.61654</lng>
<adminCode1>NE</adminCode1>
<adminName1>Nebraska</adminName1>
<adminCode2>079</adminCode2>
<adminName2>Hall</adminName2>
<adminCode3/>
<adminName3/>
</code>
<code>
<postalcode>17014<